# Stream the raw measurements and plot them

In [1]:
import serial
import struct, time
import numpy as np
import emoji
serial_port = '/dev/ttyACM0'
DATA_DIR = 'data'

In [2]:
def cal_fft(values):
    if values.shape[0] % 2 != 0:
        values = values[:-1]
    values = (values - 32767) * (3.3 / 65535.0)
    return np.absolute(np.fft.rfft(values))[1:]

def get_freqs(signal_length):
    sampling_frequency = 1000
    return np.arange(signal_length/2) * sampling_frequency / signal_length

In [3]:
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models import ColumnDataSource, Range1d, TextInput
from bokeh.models.tools import HoverTool
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import Column
output_notebook()
# output_file("lines.html")

Loading BokehJS ...

In [11]:
from threading import Thread

class Recording:
    def __init__(self, 
                 serial_dev:str = serial_port,
                 recording_name:str = '', 
                 adc_plotter = None,
                 fft_plotter = None):
        self._restart_buffers()
        self.name = recording_name
        self.adc_plotter = adc_plotter
        self.fft_plotter = fft_plotter
        self.logging_thread = None
        self.stop_threads = False
        self.serial_dev = serial_dev
    
    def _restart_buffers(self):
        self.values = np.array([], dtype=np.int64).reshape(0)
        self.indexes = np.array([], dtype=np.int64).reshape(0)
        
    def _main_logging_function(self):
        self._restart_buffers()
        period = .001  # in seconds (simulate waiting for new data)
        last_timestamp = 0
        num_reads = 100

        with serial.Serial(serial_port, 115200) as ser:
            while True:
                byts = ser.read(num_reads * 2)

                if self.stop_threads:
                    break
                    
                values = struct.unpack('{}H'.format(num_reads), byts)
                timestamp = np.linspace(last_timestamp, last_timestamp + (period*num_reads), num_reads, endpoint=False)
                last_timestamp = timestamp[-1]+period
                
                self.values = np.hstack([self.values, values])
                self.indexes = np.hstack([self.indexes, timestamp])
                
                # fft = cal_fft(values)
                # source.data = dict(
                #     time = freqs,
                #     value = fft
                # )
                
                if self.adc_plotter != None:
                    self.adc_plotter.update_plot(timestamp, values)
                if self.fft_plotter != None and self.values.shape[0] >= 250:
                    self.fft_plotter.update_plot(self.values[-250:])
                    
    
    def _flush_input(self):
        # clean serial port input buffer (ignore previous data)
        with serial.Serial(serial_port, 115200) as ser:
            ser.flushInput()
    
    def start_logging(self):
        self.stop_threads = False
        if self.logging_thread == None:
            self.logging_thread = Thread(target=self._main_logging_function)
        if self.adc_plotter:
            self.adc_plotter.init()
        if self.fft_plotter:
            self.fft_plotter.init()
        self._send_single_byte(0)
        self._send_single_byte(0)
        self._send_single_byte(0)
        self._flush_input()
        self._send_single_byte(1)
        self.logging_thread.start()
        print("logging started")
            
    def stop_logging(self):
        self.stop_threads = True
        if self.logging_thread:
            self.logging_thread.join()
        self._send_single_byte(0)
        self.logging_thread = None
        self.save_data()
        print("logging stopped")
    
    def _send_single_byte(self, byt: int):
        with serial.Serial(serial_port, 115200) as ser:
            ser.write(bytes([byt]))
    
    def save_data(self):
        timestr = time.strftime("%Y%m%d-%H%M%S")
        with open(f'{DATA_DIR}/{self.name}_{timestr}.npy','wb') as f:
            np.save(f, self.values)
            np.save(f, self.indexes)
    
    
class ADCPlotter:
    def __init__(self, n_show = 3* 1000):
        self.n_samples_to_show = n_show
    
    def init(self):
        hTool = HoverTool(
            tooltips = [("time", "@time"),("value", "@value")],
            mode='vline'
        )
        
        self.figure = figure(plot_width=900, plot_height=200)
        self.figure.y_range=Range1d(0, (2**16)-1)
        self.figure.add_tools(hTool)
        self.data_source = ColumnDataSource(data=dict(time=[], value=[]))
        self.line = self.figure.line(x='time' , 
                                     y='value', 
                                     source = self.data_source, 
                                     legend_label="ADC", 
                                     line_width=2)
        self.handle = show(self.figure, notebook_handle=True)
        
    def update_plot(self,time,value):
        new_data=dict(time=[], value=[])
        new_data['time'] = time
        new_data['value'] = value
        self.data_source.stream(new_data, self.n_samples_to_show)
        push_notebook(handle=self.handle)


class FFTPlotter:
    def __init__(self): pass
    
    def init(self):
        hTool = HoverTool(
            tooltips = [("frequency", "@freq"),("fft", "@value"), ("index", '@index')],
            mode='vline'
        )
        
        self.figure = figure(plot_width=900, plot_height=200)
        self.figure.y_range=Range1d(0, 20)
        self.figure.add_tools(hTool)
        self.data_source = ColumnDataSource(data=dict(freq=[], value=[], index =[]))
        self.line = self.figure.line(x='freq' , 
                                     y='value', 
                                     source = self.data_source, 
                                     legend_label="fft", 
                                     line_width=2)
        self.text_input = TextInput(value='', title='')
        self.widg = Column(self.text_input, self.figure)
        self.handle = show(self.widg, notebook_handle=True)
    
    def update_plot(self,values):
        new_data=dict(freq=[], value=[], index=[])
        fft = cal_fft(values)
        new_data['freq'] = get_freqs(fft.shape[0]*2)
        new_data['value'] = fft
        new_data['index'] = np.arange(fft.shape[0])
        self.check_fft(new_data)
        self.data_source.data = new_data
        push_notebook(handle=self.handle)
    
    
    def check_fft(self, new_data):
        idxs = np.logical_and(new_data['freq'] > 75 , new_data['freq'] < 150)
        num_freq_passing_th = np.sum(new_data['value'][idxs] > 6)
        self.text_input.value = f"{num_freq_passing_th}"
        if num_freq_passing_th != 0:
#             print(num_freq_passing_th)
            print(emoji.emojize('Too much vibration :vibration_mode:, Be careful, We :heart_decoration: our neighbors'))

In [15]:
recording = Recording(recording_name= 'TestAlarming',
                      adc_plotter = ADCPlotter(),
                      fft_plotter = FFTPlotter()
                     )

In [18]:
recording.start_logging()

logging started
Too much vibration 📳, Be careful, We 💟 our neighbors
Too much vibration 📳, Be careful, We 💟 our neighbors
Too much vibration 📳, Be careful, We 💟 our neighbors
Too much vibration 📳, Be careful, We 💟 our neighbors
Too much vibration 📳, Be careful, We 💟 our neighbors


In [19]:
recording.stop_logging()

logging stopped
